In [1]:
import GPy
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import safeopt
from typing import List, Tuple

import hyperspaces as hs
from objective_functions import bird_function, BIRD_FUNCTION_BOUNDS, BIRD_FUNCTION_THRESHOLD


warning in stationary: failed to import cython module: falling back to numpy
warning in coregionalize: failed to import cython module: falling back to numpy
warning in choleskies: failed to import cython module: falling back to numpy


In [2]:
# Measurement noise
noise_var = 0.25 ** 2

# Bounds on the inputs variable
bounds = BIRD_FUNCTION_BOUNDS
hyperspaces = hs.create_hyperspaces(bounds, 2)
parameter_set = safeopt.linearly_spaced_combinations(bounds, 1000)

# Define Kernel
kernel = GPy.kern.RBF(
    input_dim=len(bounds),
    variance=4.,
    lengthscale=1.0,
    ARD=True
)

# Initial safe set
x0 = np.zeros((1, len(bounds)))  # safe point at zero
# x0 = np.array([[-2.5]]) # 1D single safe point
# x0 = np.array([[-2.5, 3.4]]) # 2D single safe point
# x0 = np.array([[-2.5], [6.5]]) # 1D multiple safe points

# noisy true function
objective_function = bird_function

"""
IF initial safe set contains more than single point:
    group points corresponding to hyperspace,
    then deploy each hyperspace into a separate process.
ELSE
    deploy single process for whole search space
"""

# The statistical model of our objective function
gp = GPy.models.GPRegression(
    x0, objective_function(x0),
    kernel, noise_var=noise_var
)

# The optimization routine
# opt = safeopt.SafeOptSwarm(gp, -np.inf, bounds=bounds, threshold=0.2)
opt = safeopt.SafeOpt(
    gp,
    parameter_set,
    -35.,
    lipschitz=None,
    threshold=0.2
)


In [12]:
gp.predict(np.array([[-6.28318531,  3.14159265]]))
# gp.Y

(array([[-5.22354238e-11]]), array([[2.0025]]))

In [ ]:
# x0 = np.array([[-2., 2.], [-2.5, 3.4], [0.1, -3.], [2., 4.]])
x0 = np.array([[-2., 2.]])
safe_hyperspaces = {}
# if x0.shape[0] == 1:
#     # return whole space 
#     print(x0[0], bounds)
# else:
# return safe points and their respective hyperspace
for point in x0:
    for i, hyperspace in enumerate(hyperspaces):
        belongs_flag = True
        for dimension, value in enumerate(point):
            if value >= hyperspace[dimension][0] \
                and value <= hyperspace[dimension][1]:
                continue
            belongs_flag = False
        if belongs_flag:
            if i not in safe_hyperspaces:
                safe_hyperspaces[i] = [point]
            else:
                safe_hyperspaces[i].append(point)

In [ ]:
# print(list(zip(opt.x, opt.y)))
type(opt.data[0])

In [ ]:

evaluation_constraint = 10
for i in range(evaluation_constraint):
    # obtain new query point
    x_next = opt.optimize()

    """
    IF x_next belongs to same hyperspace:
        continue in same process
    ELSE
        deploy new process with x_next as initial safe point for its 
        hyperspace
        change search space of current process accordingly
    """

    # Get a measurement from the real system
    y_meas = objective_function(x_next)
    # Add this to the GP model
    opt.add_new_data_point(x_next, y_meas)

    opt.plot(100, plot_3d=False)
